<a href="https://www.kaggle.com/code/aayusmaanjain/license-plate-detection?scriptVersionId=101824514" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Importing Dependencies

In [1]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
import os 
import shutil
from tqdm import tqdm
import yaml
import matplotlib.pyplot as plt 
import torch
import pytesseract as pt
import cv2
%matplotlib inline

c:\Users\pulki\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
filenames = []

size_props = {
    'height':[],
    'width':[]
}

bounding_box_props = {
    'xmin':[],
    'ymin':[],
    'xmax':[],
    'ymax':[]
}

# Fetching required properties from annotations

In [3]:
annotations_path = 'input/car-plate-detection/annotations'
for file in tqdm(os.listdir(annotations_path)):
    annotation = ET.parse(os.path.join(annotations_path, file))
    filenames.append(os.path.join(annotations_path, file))
    size = annotation.find('size')
    for name, prop_list in size_props.items():
        prop_value = size.find(name).text
        size_props[name].append(int(prop_value))
    bounding_box = annotation.find('object').find('bndbox')
    for name, prop_list in bounding_box_props.items():
        prop_value = bounding_box.find(name).text
        bounding_box_props[name].append(int(prop_value))

100%|██████████| 433/433 [00:00<00:00, 820.71it/s]


In [4]:
df = pd.DataFrame({
    'file':filenames,
    'width':size_props['width'],
    'height':size_props['height'],
    'xmin':bounding_box_props['xmin'],
    'ymin':bounding_box_props['ymin'],
    'xmax':bounding_box_props['xmax'],
    'ymax':bounding_box_props['ymax']
})

In [5]:
df.head()

,file,width,height,xmin,ymin,xmax,ymax
0,input/car-plate-detection/annotations\Cars0.xml,500,268,226,125,419,173
1,input/car-plate-detection/annotations\Cars1.xml,400,248,134,128,262,160
2,input/car-plate-detection/annotations\Cars10.xml,400,225,140,5,303,148
3,input/car-plate-detection/annotations\Cars100.xml,400,267,175,114,214,131
4,input/car-plate-detection/annotations\Cars101.xml,400,300,167,202,240,220


# Making annotations compatible with YOLO

In [6]:
df['center_x'] = (df['xmax'] + df['xmin'])/(2*df['width'])
df['center_y'] = (df['ymax'] + df['ymin'])/(2*df['height'])

df['bb_width'] = (df['xmax'] - df['xmin'])/df['width']
df['bb_height'] = (df['ymax'] - df['ymin'])/df['height']

In [7]:
df.head()

,file,width,height,xmin,ymin,xmax,ymax,center_x,center_y,bb_width,bb_height
0,input/car-plate-detection/annotations\Cars0.xml,500,268,226,125,419,173,0.64500,0.555970,0.3860,0.179104
1,input/car-plate-detection/annotations\Cars1.xml,400,248,134,128,262,160,0.49500,0.580645,0.3200,0.129032
2,input/car-plate-detection/annotations\Cars10.xml,400,225,140,5,303,148,0.55375,0.340000,0.4075,0.635556
3,input/car-plate-detection/annotations\Cars100.xml,400,267,175,114,214,131,0.48625,0.458801,0.0975,0.063670
4,input/car-plate-detection/annotations\Cars101.xml,400,300,167,202,240,220,0.50875,0.703333,0.1825,0.060000


In [8]:
# Keeping important columns only 
yolo_df = df[['file', 'center_x', 'center_y', 'bb_width', 'bb_height']]
# Performing 70-15-15 split
test_size = int(0.15 * len(df))

df_train, df_test = train_test_split(yolo_df, test_size=test_size)
df_train, df_val = train_test_split(df_train, test_size=test_size)

In [9]:
train_path = os.path.join('Images', 'train')
val_path = os.path.join('Images','val')
test_path = os.path.join('Images', 'test')
images_path = 'input/car-plate-detection/images'

if not os.path.exists(train_path):
    os.makedirs(train_path)
    print('Made folder for train set')

if not os.path.exists(val_path):
    os.makedirs(val_path)
    print('Made folder for val set')

if not os.path.exists(test_path):
    os.makedirs(test_path)
    print('Made folder for test set')

# Creating train and test folders 

In [10]:
print('Moving images for train set')
for _, row  in tqdm(df_train.iterrows()):
    annotation_path = row['file']
    image_name = os.path.split(annotation_path)[-1].replace('.xml','')
    image_src = os.path.join(images_path, f'{image_name}.png')
    image_dst = os.path.join(train_path, f'{image_name}.png')
    shutil.copy2(image_src, image_dst)
    label_text = f"0 {row['center_x']} {row['center_y']} {row['bb_width']} {row['bb_height']}"
    with open(os.path.join(train_path, f'{image_name}.txt'), 'w') as f:
        f.write(label_text)
print('Done moving images for train set')

print('Moving images for val set')
for _, row  in tqdm(df_val.iterrows()):
    annotation_path = row['file']
    image_name = os.path.split(annotation_path)[-1].replace('.xml','')
    image_src = os.path.join(images_path, f'{image_name}.png')
    image_dst = os.path.join(val_path, f'{image_name}.png')
    shutil.copy2(image_src, image_dst)
    label_text = f"0 {row['center_x']} {row['center_y']} {row['bb_width']} {row['bb_height']}"
    with open(os.path.join(val_path, f'{image_name}.txt'), 'w') as f:
        f.write(label_text)
print('Done moving images for val set')

print('Moving images for test set')
for _, row  in tqdm(df_test.iterrows()):
    annotation_path = row['file']
    image_name = os.path.split(annotation_path)[-1].replace('.xml','')
    image_src = os.path.join(images_path, f'{image_name}.png')
    image_dst = os.path.join(test_path, f'{image_name}.png')
    shutil.copy2(image_src, image_dst)
    label_text = f"0 {row['center_x']} {row['center_y']} {row['bb_width']} {row['bb_height']}"
    with open(os.path.join(test_path, f'{image_name}.txt'), 'w') as f:
        f.write(label_text)
print('Done moving images for test set')

Moving images for train set


305it [00:03, 87.78it/s] 


Done moving images for train set
Moving images for val set


64it [00:00, 103.00it/s]


Done moving images for val set
Moving images for test set


64it [00:00, 104.68it/s]

Done moving images for test set


# Training YOLO model

In [13]:
# Cloning the ultralytics yolo repository
! git clone https://github.com/ultralytics/yolov5.git

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [14]:
# Installing the dependencies required for YOLO
! pip install -r yolov5/requirements.txt

In [15]:
# creating data.yaml file which will be used to train YOLO on custom data 
data = {
    'names':['License Plate'],
    'nc':1,
    'train':os.path.abspath(train_path),
    'val':os.path.abspath(val_path)
}

with open('data.yaml', 'w') as f:
    yaml.dump(data, f)

In [ ]:
# Training the YOLO model 
! python ./yolov5/train.py --data ./data.yaml  --batch-size 8  --epochs 100 --weights yolov5/yolov5s.pt

# Loading best model 

In [19]:
# Fetching the latest runs
yolo_path = 'yolov5/runs/train/'
latest_run = os.listdir(yolo_path)[-1]

# Fetching the best weights 
best_weights = os.path.join(yolo_path, latest_run, 'weights', 'best.pt')

# Loading the model with best weights trained on custom data 
model = torch.hub.load('ultralytics/yolov5', 'custom', best_weights)

C:\Users\pulki\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\hub.py:266: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\pulki/.cache\torch\hub\master.zip
C:\Users\pulki\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\detection\anchor_utils.py:63: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but

Exception: [Errno 2] No such file or directory: 'yolov5\\runs\\train\\exp2\\weights\\best.pt'. Cache may be out of date, try `force_reload=True` or see https://github.com/ultralytics/yolov5/issues/36 for help.

# Visualizing predictions

In [ ]:
%matplotlib inline
# Fetching random 16 images from test set
test_files = df_test['file'].apply(lambda x: x.replace('annotations','images').replace('xml','png'))
test_images = np.random.choice(test_files, size=(4,4))

# Visualizing predictions
fig, ax = plt.subplots(figsize=(16,16), nrows=4, ncols=4)
for i in range(4):
    for j in range(4):
        # Reading the image 
        image = cv2.imread(test_images[i,j])
        # Convert image to RGB colorspace
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Predicting from model
        results = model(image)
        # Getting co ordinates of license plate
        results_df = results.pandas().xyxy[0].loc[0]
        x_min = int(results_df['xmin'])
        x_max = int(results_df['xmax'])
        y_min = int(results_df['ymin'])
        y_max = int(results_df['ymax'])
        # Cropping license plate from image ""
        number_plate = image[y_min:y_max,x_min:x_max]
        # Performing OCR on image 
        text = pt.image_to_string(number_plate)
        # Showing the results 
        ax[i,j].set_title(str(text).strip())
        ax[i,j].imshow(np.squeeze(results.render()))
plt.show()

In [ ]:
import easyocr

In [ ]:
runs_path = os.path.join('yolov5', 'runs', 'train')
path = os.path.join(runs_path, latest_run, 'weights', 'best.pt')

model = torch.hub.load('ultralytics/yolov5', 'custom', path=path)
reader = easyocr.Reader(lang_list=['en'])


In [ ]:
plate_image = cv2.imread("car_image")
car_image = cv2.cvtColor(car_image, cv2.COLOR_BGR2RGB)
results = model(car_image)
final_img = np.squeeze(results.render())
try:
     # Getting co ordinates of license plate
    results_df = results.pandas().xyxy[0].loc[0]
    x_min = int(results_df['xmin'])
    x_max = int(results_df['xmax'])
    y_min = int(results_df['ymin'])
    y_max = int(results_df['ymax'])
    # Cropping license plate from image 
    number_plate = frame[y_min:y_max,x_min:x_max]
    # Converting the number plate to grayscale
    number_plate = cv2.cvtColor(number_plate, cv2.COLOR_RGB2GRAY)
    # Binarizing the image 
    _, number_plate = cv2.threshold(number_plate, 128,255, cv2.THRESH_BINARY)
    texts = reader.readtext(number_plate)
    license_plate_no = texts[0][-2] if len(texts) == 1 else ' '.join([text[-2] for text in texts])
    final_img = cv2.putText(final_img, str(license_plate_no), org=(x_min, y_max), 
                            fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1, color=(0,255,0), thickness=2)
except:
    pass
cv2.imshow('Image', final_img)
if cv2.waitKey(10) & 0xFF == ord('q'):
    break
